d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# IDBML 04b - Implementing a Webhook

<img src="https://s3.us-west-2.amazonaws.com/files.training.databricks.com/images/idbml/04b-image.png">

## Model Registry Webhooks

### Supported Events
* Registered model created
* Model version created
* **Transition request created**
* **Model version transitioned stage**

### Types of webhooks
* **HTTP webhook** &mdash; send triggers to endpoints of your choosing such as slack, AWS Lambda, Azure Functions, or GCP Cloud Functions
* Job webhook &mdash; trigger a job within the Databricks workspace

## Implementing a Webhook

In this demonstration, we're going to implement a couple of notification webhooks using HTTP endpoints and Slack.

### Helper Function

The first thing we're going to do is create a helper function to call MLflow endpoints.

In [0]:
import mlflow
from mlflow.utils.rest_utils import http_request
import json

def client():
    return mlflow.tracking.client.MlflowClient()

host_creds = client()._tracking_client.store.get_host_creds()
host = host_creds.host
token = host_creds.token

def mlflow_call_endpoint(endpoint, method, body='{}'):
    if method == 'GET':
        response = http_request(
            host_creds=host_creds, endpoint="/api/2.0/mlflow/{}".format(endpoint), method=method, params=json.loads(body)
        )
    else:
        response = http_request(
            host_creds=host_creds, endpoint="/api/2.0/mlflow/{}".format(endpoint), method=method, json=json.loads(body)
        )
    return response.json()

-sandbox
### Setting Up Slack Notifications

Webhooks can be used to send emails, Slack messages, and more. In this case, we demonstrate the use **Slack for notification purposes**.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> You'll need to set up the Slack webhook yourself within Slack, but you can read more about Slack webhooks [here](https://api.slack.com/messaging/webhooks#create_a_webhook).

#### Transition Request Notification

First, we set up a webhook to notify us whenever a **Model Registry transition request is created**.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> We've hidden the Slack webhook [here]($./Includes/Slack-Webhook) for security purposes, but it should take the form of: `"https://hooks.slack.com/services/T????????/B?????????/????????????????????????"`

In [0]:
%run ./Includes/Slack-Webhook

In [0]:
import json 

model_name = "idbml-airbnb-price"

trigger_slack = json.dumps({
    "model_name": model_name,
    "events": ["TRANSITION_REQUEST_CREATED"],
    "description": "This notification triggers when a model is requested to be transitioned to a new stage.",
    "status": "ACTIVE",
    "http_url_spec": {
        "url": slack_webhook
    }
})

mlflow_call_endpoint("registry-webhooks/create", method = "POST", body = trigger_slack)

Let's test it out by requesting to transition the model to the **Production** stage.

#### Transition Notification

Rather than triggering on a request, this notification will trigger a Slack message when a model is successfully transitioned to a new stage.

In [0]:
import json 

trigger_slack = json.dumps({
  "model_name": model_name,
  "events": ["MODEL_VERSION_TRANSITIONED_STAGE"],
  "description": "This notification triggers when a model is transitioned to a new stage.",
  "http_url_spec": {
    "url": slack_webhook
  }
})

mlflow_call_endpoint("registry-webhooks/create", method = "POST", body = trigger_slack)

Let's test it out by transitioning the model to the **Prodiction** stage.

In [0]:
mlflow_client = mlflow.tracking.client.MlflowClient()
model_version = int(dict(mlflow_client.get_latest_versions(model_name)[0])["version"])
mlflow_client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage="Production",
    archive_existing_versions=True
)

-sandbox
### Manage Webhooks

You can manage your webhooks pretty easily, too.

First, you can list all of your webhooks and corresponding information using the cell below.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Notice that these are model-specific webhooks to keep from interfering with others' workflows.

In [0]:
list_model_webhooks = json.dumps({"model_name": model_name})

model_webhooks = mlflow_call_endpoint("registry-webhooks/list", method = "GET", body = list_model_webhooks)
model_webhooks

You can also **delete webhooks**.

You can use the below cell to delete webhooks by ID.

In [0]:
mlflow_call_endpoint(
    "registry-webhooks/delete",
    method="DELETE",
    body=json.dumps({'id': model_webhooks["webhooks"][0]["id"]})
)

Or you can use the below cell to delete all webhooks for a specific model.

In [0]:
for webhook in model_webhooks["webhooks"]:
    mlflow_call_endpoint(
    "registry-webhooks/delete",
    method="DELETE",
    body=json.dumps({'id': webhook["id"]})
)

And finally, verify that they're all deleted.

In [0]:
updated_model_webhooks = mlflow_call_endpoint("registry-webhooks/list", method = "GET", body = list_model_webhooks)
updated_model_webhooks


-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>